## Import libraries needed for this project

In [22]:
#!pip install seaborn
#!pip install scipy
#!pip install statsmodels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

  Using cached statsmodels-0.14.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.2 kB)
  Using cached patsy-0.5.6-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached statsmodels-0.14.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.7 MB)
Using cached patsy-0.5.6-py2.py3-none-any.whl (233 kB)


## Data Collection

### The data used in this project is from kaggle (https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset?resource=download). 

In [3]:
## read the data from csv file
df = pd.read_csv('hotel.csv')
df

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.50,0,Canceled


## Data Cleaning

In [3]:
## see whether there are missing data
df.loc[np.sum(df.isna(),axis = 1)>0,:]

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status


In [4]:
## rename the columns: shorten the names of some columns, such as required_car_parking_space = parking; room_type_reserved = room_type; etc.
column_rename_map = {
    'no_of_weekend_nights': 'weekend_nights',
    'no_of_week_nights': 'week_nights',
    'type_of_meal_plan': 'meal_plan',
    'required_car_parking_space': 'parking',
    'room_type_reserved': 'room_type',
    'market_segment_type':'market',
    'no_of_previous_cancellations':'prev_cancellations',
    'no_of_previous_bookings_not_canceled': 'prev_bookings',
    'avg_price_per_room': 'avg_price',
    'no_of_special_requests': 'special_requests'
}
df = df.rename(columns=column_rename_map)
df.head()

,Booking_ID,no_of_adults,no_of_children,weekend_nights,week_nights,meal_plan,parking,room_type,lead_time,arrival_year,arrival_month,arrival_date,market,repeated_guest,prev_cancellations,prev_bookings,avg_price,special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


#### Convert categorical data using dummy variables

In [13]:
# meal_plan column
#df['meal_plan'].unique()
mealplan_dict = {
    'Not Selected': 0,
    'Meal Plan 1': 1,
    'Meal Plan 2': 2,
    'Meal Plan 3': 3
}
df['meal'] = df['meal_plan'].map(lambda x: mealplan_dict[x])

In [15]:
# room_type column
#df['room_type'].unique()
room_map = {
    'Room_Type 1': 1,
    'Room_Type 2': 2,
    'Room_Type 3': 3,
    'Room_Type 4': 4,
    'Room_Type 5': 5,
    'Room_Type 6': 6,
    'Room_Type 7': 7
}
df['room'] = df['room_type'].map(lambda x: room_map[x])

In [18]:
# market column
#df['market'].unique()
market_map = {
    'Offline': 0,
    'Online': 1,
    'Corporate': 2,
    'Aviation': 3,
    'Complementary': 4
}
df['mkt'] = df['market'].map(lambda x: market_map[x])

In [34]:
# booking status column (for ANOVA test 1 below)
booking_map = {
    'Not_Canceled': 0,
    'Canceled': 1
}
df['cancel'] = df['booking_status'].map(lambda x: booking_map[x])

In [35]:
df['no_of_children'].unique()

array([ 0,  2,  1,  3, 10,  9])

In [37]:
# no_of_children column (for ANOVA test 2 below)
child_map = {
    0: 0,
    1: 1,
    2: 1,
    3: 1,
    9: 1,
    10: 1
}
df['bring_child'] = df['no_of_children'].map(lambda x: child_map[x])

In [42]:
# create a season column based on the value of arrival_month column (spring: 1; summer: 2; fall: 3; winter: 4)
def map_season(month):
    if month >= 3 and month <= 5:
        return 1
    elif month >= 6 and month <= 8:
        return 2
    elif month >= 9 and month <= 11:
        return 3
    elif month == 12 or month <= 2:
        return 4
    else:
        return 'error'
df['season'] = df['arrival_month'].apply(map_season)

#### Examine whether there are outliers (calculate mean,median,std,etc. of continuous data)

In [5]:
## price column
avg_p = np.mean(df['avg_price'])
median_p = np.median(df['avg_price'])
std_p = np.std(df['avg_price'])
print('The mean, median, and standard deviation of column avg_price are: ',avg_p,median_p,std_p)

The mean, median, and standard deviation of column avg_price are:  103.42353907649897 99.45 35.08894037453894


## Data Analysis

In [29]:
## Are the cancel rates for customers with different room types different statistically? 
room_ANOVA = ols('cancel ~ room', data=df).fit()
anova_table1 = sm.stats.anova_lm(room_ANOVA, typ=2)
print(anova_table)

               sum_sq       df          F    PR(>F)
room         4.222254      1.0  19.175814  0.000012
Residual  7986.822543  36273.0        NaN       NaN


#### As shown in the table above, the p-value for room (room_type) variables is 0.000012, which is way less than 0.05, therefore, we can conclude that the cancel rates for customers with different room types are statistically inconsistent (different).

In [28]:
## Now, let's see whether the room type is correlated with booking status, and if so, the direction of such correlation
pearsonr_room = df['room'].corr(df['cancel'])
print("Pearson's correlation coefficient:", pearsonr_room)

Pearson's correlation coefficient: 0.0229863701866078


#### A Pearson's r of 0.0229863701866078 indicates that there is a weak positive correlation between room type and booking status, in spite of the inconsistent cancel rates among customers with different room types.

In [30]:
## Do those who bring their children with them honor their reservation more than those who do not?
child_ANOVA = ols('cancel ~ no_of_children', data=df).fit()
anova_table2 = sm.stats.anova_lm(child_ANOVA, typ=2)
print(anova_table2)

                     sum_sq       df          F        PR(>F)
no_of_children     8.743340      1.0  39.731293  2.947637e-10
Residual        7982.301457  36273.0        NaN           NaN


#### As shown in the table above, the p-value for children (no_of_children) variables is 2.947e-10, which is way less than 0.05, therefore, we can conclude that the cancel rates for customers who bring different numbers of children with them are statistically unconsistent (different).

In [32]:
## Now, let's see whether the no_of_children is correlated with booking status, and if so, the direction of such correlation
pearsonr_child = df['no_of_children'].corr(df['cancel'])
print("Pearson's correlation coefficient:", pearsonr_child)

Pearson's correlation coefficient: 0.03307782091225096


#### A Pearson's r of 0.022078 indicates that there is a weak positive correlation between number of children and booking status, in spite of the inconsistent cancel rates among customers who bring different numbers of children with them.

#### Meanwhile, we wonder whether simply categorizing customers into two groups based on whether they bring children with them (instead of based on the number of child they bring) would yield different results.

In [38]:
pearsonr_children = df['bring_child'].corr(df['cancel'])
print("Pearson's correlation coefficient:", pearsonr_children)

Pearson's correlation coefficient: 0.02664678277572277


#### A Pearson's r of 0.02665 indicates that there is a weak positive correlation between whether a customer brings child and whether the customer cancels the booking.

In [39]:
## Is there seasonality for cancel rate? (Whether cancel rate are inconsistent throughout a year) 
month_ANOVA = ols('cancel ~ arrival_month', data=df).fit()
anova_table3 = sm.stats.anova_lm(month_ANOVA, typ=2)
print(anova_table3)

                    sum_sq       df         F  PR(>F)
arrival_month     1.008322      1.0  4.577559  0.0324
Residual       7990.036475  36273.0       NaN     NaN


#### As shown in the table above, the p-value for month (arrival_month) variables is 0.0324, which is less than 0.05, therefore, we can conclude that the cancel rates for customers arrived in different months of a year are statistically inconsistent (different).

In [40]:
## Now, let's see whether the room type is correlated with booking status, and if so, the direction of such correlation
pearsonr_month = df['arrival_month'].corr(df['cancel'])
print("Pearson's correlation coefficient:", pearsonr_month)

Pearson's correlation coefficient: -0.011233054158994645


#### A Pearson's r of -0.011233 indicates that there is a weak negative correlation between arrival month and booking status, in spite of the inconsistent cancel rates among customers who arrived in different months of a year.

#### Meanwhile, we wonder whether categorizing customers into 4 groups based on the season they arrived (spring,summer,fall,winter) would yield different results.

In [43]:
pearsonr_season = df['season'].corr(df['cancel'])
print("Pearson's correlation coefficient:", pearsonr_season)

Pearson's correlation coefficient: -0.12447779496289736


#### A Pearson's r of -0.1245 indicates that there is a weak negative correlation between arrival season and booking status.

In [ ]:
## Is the cancel rate in 2017 different from 2018? 

In [ ]:
## Examine whether the data complies with the assumptions for modeling (logistics regression)

## Data Visualization (draw it when needed)

## Modeling

In [ ]:
## predictive model for whether a customer, given information about the booking, would honor or cancel the reservation